# Yêu cầu
Cho một tập hợp các điểm trong không gian 2D (mỗi điểm có tọa độ $P = (x, y)$), hãy tìm ra hai điểm có khoảng cách gần nhau nhất.

### Phương pháp "Vét cạn" (Brute Force)

In [1]:
import math

def calculate_distance_sq(p1, p2):
    """Tính bình phương khoảng cách giữa hai điểm (x, y)."""
    return (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2

def find_closest_pair_brute_force(points):
    """
    Tìm cặp điểm gần nhất bằng phương pháp "vét cạn".
    Input: points - một danh sách các tuple (x, y).
    """
    if len(points) < 2:
        return None, float('inf') # Không thể tìm cặp nếu có ít hơn 2 điểm

    # Khởi tạo giá trị ban đầu
    min_dist_sq = float('inf') # Dùng bình phương khoảng cách
    closest_pair = (None, None)

    # Vòng lặp i từ điểm 0 đến điểm n-2
    for i in range(len(points)):
        # Vòng lặp j từ điểm i+1 đến điểm n-1
        for j in range(i + 1, len(points)):
            p1 = points[i]
            p2 = points[j]
            
            dist_sq = calculate_distance_sq(p1, p2)
            
            # Nếu tìm thấy khoảng cách nhỏ hơn
            if dist_sq < min_dist_sq:
                min_dist_sq = dist_sq
                closest_pair = (p1, p2)
                
    # Tính căn bậc hai CHỈ MỘT LẦN ở cuối
    min_distance = math.sqrt(min_dist_sq)
    
    return closest_pair, min_distance

# --- Ví dụ sử dụng ---
list_of_points = [(1, 1), (4, 1), (3, 3), (7, 5), (1, 6), (3, 4)]

pair, distance = find_closest_pair_brute_force(list_of_points)

print(f"Danh sách điểm: {list_of_points}")
print(f"Cặp điểm gần nhất là: {pair}")
print(f"Khoảng cách: {distance:.4f}") 
# Kết quả mong đợi: Cặp (3, 3) và (3, 4), khoảng cách 1.0

Danh sách điểm: [(1, 1), (4, 1), (3, 3), (7, 5), (1, 6), (3, 4)]
Cặp điểm gần nhất là: ((3, 3), (3, 4))
Khoảng cách: 1.0000


### Phương pháp "Chia để trị" (Divide and Conquer)
- Ý tưởng logicTư tưởng chính là: "Chia một vấn đề lớn thành các vấn đề nhỏ hơn, giải các vấn đề nhỏ, rồi kết hợp kết quả lại."
- Sắp xếp (Sort): Bước đầu tiên và quan trọng nhất: Sắp xếp tất cả các điểm theo tọa độ $x$.
- Chia (Divide): Tìm một đường thẳng đứng (ví dụ, đi qua điểm ở giữa) chia tập hợp điểm thành hai nửa bằng nhau: Nửa Trái và Nửa Phải.
- Trị (Conquer): Đệ quy!
        - Tìm cặp gần nhất trong Nửa Trái. Gọi khoảng cách nhỏ nhất ở đây là $\delta_L$ (delta-left).
        - Tìm cặp gần nhất trong Nửa Phải. Gọi khoảng cách nhỏ nhất ở đây là $\delta_R$ (delta-right).
- Kết hợp (Combine):Khoảng cách nhỏ nhất chúng ta biết cho đến nay là $\delta = \min(\delta_L, \delta_R)$.Đây là phần "khó" nhất: Sẽ ra sao nếu cặp gần nhất lại là một điểm ở Nửa Trái và một điểm ở Nửa Phải? Chúng ta không thể so sánh tất cả các điểm bên trái với tất cả các điểm bên phải (vì như vậy lại quay về $O(n^2)$).
- Tối ưu hóa: Chúng ta chỉ cần quan tâm đến các điểm nằm rất gần đường ranh giới. Cụ thể, chúng ta chỉ cần xét các điểm nằm trong một "dải" (strip) có chiều rộng $2\delta$ (tức là $\delta$ về bên trái và $\delta$ về bên phải của đường ranh giới).Tập hợp các điểm trong dải này (gọi là strip_points) được sắp xếp theo tọa độ $y$.
- Một chứng minh toán học (khá phức tạp) chỉ ra rằng: với mỗi điểm $P$ trong dải này, bạn chỉ cần so sánh nó với một số lượng hằng định các điểm phía sau nó (ví dụ: 7-8 điểm tiếp theo trong danh sách đã sắp xếp theo $y$), chứ không phải tất cả.
- Nếu tìm thấy một cặp trong "dải" này có khoảng cách nhỏ hơn $\delta$, ta cập nhật $\delta$.Kết quả cuối cùng chính là $\delta$.
- Đánh giá:
        - Ưu điểm: Rất nhanh, $O(n \log n)$. Đây là cách tiêu chuẩn để giải quyết vấn đề này trong các cuộc thi lập trình hoặc với dữ liệu lớn.
        - Nhược điểm: Cài đặt phức tạp hơn đáng kể, đặc biệt là ở bước "Combine" (xử lý dải ở giữa).

In [ ]:
import math

# --- Hàm helper 1: Tính bình phương khoảng cách ---
def calculate_distance_sq(p1, p2):
    """Tính bình phương khoảng cách giữa hai điểm (x, y)."""
    return (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2

# --- Hàm helper 2: Vét cạn cho trường hợp cơ sở ---
def brute_force_small(points):
    """
    Hàm vét cạn (Cách 1) nhưng chỉ dùng cho trường hợp cơ sở (n <= 3).
    Trả về (cặp_điểm, khoảng_cách_bình_phương).
    """
    min_dist_sq = float('inf')
    closest_pair = (None, None)
    
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            dist_sq = calculate_distance_sq(points[i], points[j])
            if dist_sq < min_dist_sq:
                min_dist_sq = dist_sq
                closest_pair = (points[i], points[j])
                
    return closest_pair, min_dist_sq

# --- Hàm 3: Hàm đệ quy cốt lõi ---
def find_closest_pair_dc_recursive(points_sorted_x):
    """
    Hàm đệ quy thực hiện logic Chia-Trị-Kết hợp.
    Input: points_sorted_x - danh sách các điểm ĐÃ ĐƯỢC sắp xếp theo x.
    Output: (cặp_điểm, khoảng_cách_bình_phương).
    """
    n = len(points_sorted_x)
    
    # === BƯỚC 1: TRƯỜNG HỢP CƠ SỞ (BASE CASE) ===
    # Nếu chỉ còn 3 điểm hoặc ít hơn, dùng vét cạn là nhanh nhất.
    # Đây là điểm dừng của đệ quy.
    if n <= 3:
        return brute_force_small(points_sorted_x)

    # === BƯỚC 2: CHIA (DIVIDE) ===
    # Tìm điểm giữa để chia đôi
    mid_index = n // 2
    mid_point = points_sorted_x[mid_index]
    mid_x = mid_point[0] # Đường ranh giới dọc
    
    # Chia danh sách đã sắp xếp theo x thành 2 nửa (O(n))
    left_half_x = points_sorted_x[:mid_index]
    right_half_x = points_sorted_x[mid_index:]
    
    # === BƯỚC 3: TRỊ (CONQUER) ===
    # Gọi đệ quy cho từng nửa
    (pair_L, dist_sq_L) = find_closest_pair_dc_recursive(left_half_x)
    (pair_R, dist_sq_R) = find_closest_pair_dc_recursive(right_half_x)
    
    # === BƯỚC 4: KẾT HỢP (COMBINE) ===
    
    # 4a. Tìm delta từ hai nửa
    min_dist_sq = dist_sq_L
    closest_pair = pair_L
    
    if dist_sq_R < min_dist_sq:
        min_dist_sq = dist_sq_R
        closest_pair = pair_R
        
    # Đây là khoảng cách delta bình phương
    # delta là khoảng cách nhỏ nhất tìm được ở Trái hoặc Phải
    delta = math.sqrt(min_dist_sq) 

    # 4b. Xử lý "dải" (strip) ở giữa
    # 
    # Ta cần tìm các điểm nằm trong dải [mid_x - delta, mid_x + delta]
    strip = []
    for point in points_sorted_x:
        if abs(point[0] - mid_x) < delta:
            strip.append(point)
            
    # 4c. Sắp xếp các điểm trong dải này theo tọa độ Y
    # Đây là bước then chốt và là nơi tốn nhiều thời gian nhất trong bước "Kết hợp"
    # (Độ phức tạp O(n log n) cho bước này)
    strip.sort(key=lambda p: p[1])
    
    # 4d. Quét dải
    # Chúng ta chỉ cần so sánh mỗi điểm với một vài điểm ngay sau nó
    # (Chứng minh toán học là tối đa 7-8 điểm)
    for i in range(len(strip)):
        # j bắt đầu từ i + 1
        for j in range(i + 1, len(strip)):
            # Tối ưu quan trọng:
            # Nếu khoảng cách giữa strip[j] và strip[i] đã >= delta,
            # thì không cần xét j và các điểm sau nó nữa, vì chúng
            # chắc chắn xa hơn delta.
            if (strip[j][1] - strip[i][1]) >= delta:
                break
                
            # Nếu điểm nằm trong "tầm ngắm", tính khoảng cách
            dist_sq = calculate_distance_sq(strip[i], strip[j])
            
            # Nếu tìm thấy cặp mới tốt hơn -> cập nhật
            if dist_sq < min_dist_sq:
                min_dist_sq = dist_sq
                closest_pair = (strip[i], strip[j])
                # Cập nhật delta để vòng lặp 4d chạy chính xác hơn?
                # KHÔNG. Delta đã được cố định để xác định 'strip'.
                # Chúng ta chỉ cập nhật min_dist_sq.
                # (Chỉnh sửa: delta cũng nên được cập nhật
                # để tối ưu hóa `break` ở trên)
                delta = math.sqrt(min_dist_sq)


    # === BƯỚC 5: TRẢ VỀ ===
    # Trả về cặp gần nhất và khoảng cách bình phương nhỏ nhất tìm được
    return closest_pair, min_dist_sq

# --- Hàm 1: Hàm "bao bọc" (Wrapper) ---
def find_closest_pair_dc(points):
    """
    Hàm bao bọc chính. Sắp xếp các điểm theo X và gọi hàm đệ quy.
    Trả về (cặp_điểm, khoảng_cách_thực).
    """
    if len(points) < 2:
        return None, float('inf')
        
    # --- Bước 0: Sắp xếp theo X MỘT LẦN DUY NHẤT ---
    points_sorted_x = sorted(points, key=lambda p: p[0])
    
    # Gọi hàm đệ quy
    (pair, min_dist_sq) = find_closest_pair_dc_recursive(points_sorted_x)
    
    # Trả về kết quả cuối cùng (tính căn bậc 2 một lần duy nhất)
    return pair, math.sqrt(min_dist_sq)

# --- Ví dụ sử dụng ---
list_of_points = [(1, 1), (4, 1), (3, 3), (7, 5), (1, 6), (3, 4), (2.9, 3.1)]

pair, distance = find_closest_pair_dc(list_of_points)

print(f"Danh sách điểm: {list_of_points}")
print(f"Cặp điểm gần nhất (Chia để trị) là: {pair}")
print(f"Khoảng cách: {distance:.4f}")

# Kết quả mong đợi: Cặp (3, 3) và (2.9, 3.1), khoảng cách ~0.1414

Danh sách điểm: [(1, 1), (4, 1), (3, 3), (7, 5), (1, 6), (3, 4), (2.9, 3.1)]
Cặp điểm gần nhất (Chia để trị) là: ((3, 3), (2.9, 3.1))
Khoảng cách: 0.1414
